In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
#crate paths for csv files
CURR_DIR = os.getcwd()
print(CURR_DIR)
zlecenia_path = os.path.join(CURR_DIR, 'Zlecenia.csv')
godziny_path = os.path.join(CURR_DIR, 'Godziny.csv')

C:\Users\ksobo\Desktop\DS_final_project


In [3]:
#Zaciagniecie baz danych
obciazenie_df = pd.read_csv(zlecenia_path, sep=';')
godziny_df = pd.read_csv(godziny_path, sep=';')

In [4]:
#Ekstrakcja danych i organizacja danych - pozostawienie tylko kolumn wymaganych, pozostawienie tylko projektów "w toku"
obciazenie_df1 = obciazenie_df.drop(columns=['work_preparation_Plan','work_preparation_done'], axis=1)
obciazenie_df2 = obciazenie_df1.loc[obciazenie_df1['Active'].isin(['Active'])]

In [5]:
#Left join po No z godzinami
obciazenie_df2 = pd.merge(obciazenie_df2, godziny_df, how='left', on='No')

#wypelnienie NaN zerami
obciazenie_df2['Wszystkie.technol'] = obciazenie_df2['Wszystkie.technol'].fillna(0)
obciazenie_df2['Wszystkie.wykonane'] = obciazenie_df2['Wszystkie.wykonane'].fillna(0)

In [6]:
#Zdefiniowanie tygodnia obecnego
#przyjmujemy obecny jako 202112(tydzień zaciągnięcia danych),
#docelowo może zostać zamieniony funkcją obliczającą jak poniżej
#WPISAĆ FUNKCJĘ!!!!!!!!!

current_week = 202112


In [7]:
#Zmiana terminu rozpoczęcia oraz zakończenia na obecny jeżeli miało mieć ono miejsce w przeszłości
obciazenie_df2.loc[obciazenie_df2['Finnish_week'] < current_week, 'Finnish_week'] = current_week

obciazenie_df2.loc[obciazenie_df2['Pre_fabrication_plan'] < current_week, 'Pre_fabrication_plan'] = current_week

In [8]:
#Zlecenia które nie zostały opracowane wypełnione zostaną wartościami przewidzianymi w prekalkulacji

#A - dziób(wędka)
#Labor rate	130 rbg/1000kg
#Assembling	0,4
#Welding	0,33
#Grindining	0,07
#Else		0,2

#B - belka(środek)
#Labor rate	100 rbg/1000kg
#Assembling	0,3
#Welding	0,4
#Grindining	0,2
#Else		0,1

#C - gniazdo(końcówka)
#Labor rate	120 rbg/1000kg
#Assembling	0,25
#Welding	0,4
#Grindining	0,2
#Else		0,15

obciazenie_df2.loc[obciazenie_df2['Typ'] == 'A', 'Typ'] = 0.13
obciazenie_df2.loc[obciazenie_df2['Typ'] == 'B', 'Typ'] = 0.1
obciazenie_df2.loc[obciazenie_df2['Typ'] == 'C', 'Typ'] = 0.12

# IF Wszystkie technol = NaN -> Weight for 1 * Quantity * Typ

obciazenie_df2.loc[obciazenie_df2['Wszystkie.technol'] == 0, 'Wszystkie.technol'] = obciazenie_df2['Weight_for_1'] * obciazenie_df2['Quantity'] * obciazenie_df2['Typ']

In [9]:
#Obliczenie kolumn - pozostałego czasu na produkcję oraz pozostałych godzin
obciazenie_df2['Weeks_left'] = (obciazenie_df2['Finnish_week']) - (obciazenie_df2['Pre_fabrication_plan'])

#Jeżeli >52 to należy odjąc od wyniku 48 (100 - (ilość tyg w danym roku - def 52))
#Może tak się stać dla projektów na przełomie roku 202206-202146 = 60!
obciazenie_df2.loc[obciazenie_df2['Weeks_left'] > 52, 'Weeks_left'] = obciazenie_df2['Weeks_left'] - 48

#Jeżeli zlecenie jest opóźnione i wynik pozostałego czasu będzie wynosił 0
#to zostanie on doliczony do obecnego tygodnia pozdzielony przez 1
obciazenie_df2.loc[obciazenie_df2['Weeks_left'] < 1, 'Weeks_left'] = 1

#funkcja if niestety nie znalazła odpowiedniego zastosowania - loc jest polecany jako dobra praktyka

In [10]:
#???????Czy liczyć nadwyżkę 20% jako zabezpieczenie przy przekroczeniach??????
obciazenie_df2['Work_h_left'] = (obciazenie_df2['Wszystkie.technol']) - (obciazenie_df2['Wszystkie.wykonane'])

#Jeżeli zlecenie zlecenie przekroczyło godziny praca liczona jest w nadgodzinach - przyjmuje się 0 do obciażenia
obciazenie_df2.loc[obciazenie_df2['Work_h_left'] < 0, 'Work_h_left'] = 0

In [11]:
#Podzielenie pozostałej pracy równomiernie na tygodnie
obciazenie_df2['Workload_per_week'] = (obciazenie_df2['Work_h_left']) / (obciazenie_df2['Weeks_left'])
obciazenie_df2['Workload_per_week']

0           0
1           0
2           0
3     7.52941
4        64.7
       ...   
58         40
59         80
60         64
61         80
62         80
Name: Workload_per_week, Length: 63, dtype: object

In [12]:
#Stworzenie finalnej tabeli
obciazenie_df3 = obciazenie_df2[["No", "Pre_fabrication_plan", "Finnish_week", "Workload_per_week"]]

#Stworzenie tabeli z numerami tygodni objętych analizą
x = current_week

Weeks = np.arange(14)
Weeks = current_week
Week_by_week = np.arange(14)
Weeks += Week_by_week
Weeks = Weeks.astype(int)

Capacity = obciazenie_df3.reindex(columns = Weeks)

#Połączenie tabel w jedną
#Ustawienie kolejności kolumn
#Zamiana Nan na 0

obciazenie_df4 = pd.concat([Capacity, obciazenie_df3])
obciazenie_df4 = obciazenie_df4.fillna(0)
obciazenie_df4 = obciazenie_df4.loc[obciazenie_df4["No"].isin([0]) == False]
obciazenie_df4 = obciazenie_df4.astype(int)

# nie działa indexowanie
# obciazenie_df4.set_index("No")


In [13]:
#Podział obciążenia na tygodnie obecny+13 tyg

#można dać każdą kolumę osobno i potem to zlepić

# Dodanie obciazenia zarezerwowanego na dany tydzien w zależności od rozpoczecia i zakonczenia danego zlecenia
#If prefabrication plan is >column index & <= finnish week -> column index.No zlecenia = Workload_per_week

#### Może załatwić to iloc 'iem???
#obciazenie_df4.loc[obciazenie_df4.iloc[1] >= ['Pre_fabrication_plan'], obciazenie_df4.iloc[1]] = ['Workload_per_week']
#obciazenie_df4.loc[obciazenie_df4.iloc[1] > ['Finnish_week'], obciazenie_df4.iloc[1]] = 0

#najpierw standardowo
#obciazenie_df4.loc[current_week >= obciazenie_df4['Pre_fabrication_plan'], obciazenie_df4[202112]] = ['Workload_per_week']

#możliwość kolejna - dać tydzień w którym dane zlecenie występuje binarnie czyli 0 lub 1  i potem przemnażać przez ['Workload_per_week']
obciazenie_df4.loc[obciazenie_df4['Pre_fabrication_plan'] <= current_week, obciazenie_df4.iloc[1]] = 1
obciazenie_df4.loc[obciazenie_df4['Finnish_week'] < current_week, obciazenie_df4.iloc[1]] = 0

#Zrobić loopa i=[0:13]
#obciazenie_df4.loc[obciazenie_df4['Pre_fabrication_plan'] <= current_week+i, obciazenie_df4.iloc[1+i]] = 1
#obciazenie_df4.loc[obciazenie_df4['Finnish_week'] < current_week+i, obciazenie_df4.iloc[1+i]] = 0

obciazenie_df4

,202112,202113,202114,202115,202116,202117,202118,202119,202120,202121,...,202123,202124,202125,Finnish_week,No,Pre_fabrication_plan,Workload_per_week,0,192,1.0
0,1,0,0,0,0,0,0,0,0,0,...,1,0,0,202112,191,202112,0,1.0,1.0,NaN
1,1,0,0,0,0,0,0,0,0,0,...,1,0,0,202123,192,202112,0,1.0,1.0,NaN
2,1,0,0,0,0,0,0,0,0,0,...,1,0,0,202146,870,202112,0,1.0,1.0,NaN
3,1,0,0,0,0,0,0,0,0,0,...,1,0,0,202146,908,202112,7,1.0,1.0,NaN
4,1,0,0,0,0,0,0,0,0,0,...,1,0,0,202112,1015,202112,64,1.0,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,0,0,0,0,0,0,0,0,0,0,...,0,0,0,202234,1268,202218,40,NaN,NaN,NaN
59,0,0,0,0,0,0,0,0,0,0,...,0,0,0,202244,1269,202236,80,NaN,NaN,NaN
60,0,0,0,0,0,0,0,0,0,0,...,0,0,0,202307,1270,202249,64,NaN,NaN,NaN
61,0,0,0,0,0,0,0,0,0,0,...,0,0,0,202319,1271,202311,80,NaN,NaN,NaN


In [14]:
#Zrobienie wykresu